In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import nibabel as nb
from scipy import ndimage
import scipy.stats as ss
from joblib import Parallel, delayed

In [3]:
df = pd.read_excel('/Users/jayantadey/Human.parcellated_thickness.xlsx')

In [4]:
df.head()

,Unnamed: 0,sid,Markov.1,Markov.2,Markov.3,Markov.4,Markov.5,Markov.6,Markov.7,Markov.8,...,Schaefer217.191,Schaefer217.192,Schaefer217.193,Schaefer217.194,Schaefer217.195,Schaefer217.196,Schaefer217.197,Schaefer217.198,Schaefer217.199,Schaefer217.200
0,0,sub-OAS30876MRD4592,1.995032,2.203564,1.651978,1.969754,2.603206,2.295727,2.385144,2.719692,...,8.193966,7.736098,7.404804,7.431338,7.541022,7.433447,7.475594,7.460332,7.476401,7.609443
1,1,sub-HBN_CBIC_NDARXC962XNK,2.557198,2.011555,2.175673,1.863080,2.473705,2.576267,2.392282,2.242582,...,6.927265,7.487809,7.098700,6.753360,6.841211,6.707229,7.189156,6.795299,6.823550,6.533783
2,2,sub-AOMIC_0770,2.246607,2.295872,1.978412,2.069700,2.213602,2.449572,2.541624,2.777280,...,8.117053,7.775745,7.608771,7.579930,7.573511,7.607256,7.883723,7.630075,7.670835,7.354955
3,3,sub-AOMIC_0344,2.219745,2.366237,2.036068,2.173696,2.508847,2.408997,2.430510,2.882698,...,7.908504,7.856069,7.756918,7.526684,7.410575,7.654072,7.952062,7.682724,7.444550,7.697996
4,4,sub-Narratives_150,2.131236,2.432549,2.066158,2.352589,2.331565,2.799966,2.490590,2.818574,...,8.229232,7.847060,7.821367,7.739102,8.001608,7.803380,7.750091,7.730715,7.858832,7.892026


In [5]:
df_sex = pd.read_excel('/Users/jayantadey/data_MRI/subjects_age_sex_data_MRI.xlsx')
df_sex.head()

,ID,Age,Sex,Dataset,Dataset-ID
0,sub-ABIDE1050339,18.0000,MALE,ABIDE,50339
1,sub-ABIDE1050701,18.0000,MALE,ABIDE,50701
2,sub-ABIDE1050445,18.1383,MALE,ABIDE,50445
3,sub-ABIDE1050459,18.1547,MALE,ABIDE,50459
4,sub-ABIDE1050341,18.2000,FEMALE,ABIDE,50341


In [6]:
X1 = []
X2 = []
y_human = []
IDs = set(df['sid'])
ref_IDs = set(df_sex['ID'])

for subject in tqdm(IDs):
    if subject in ref_IDs:
        features = np.array(df[df['sid']==subject]).reshape(-1)[2:]
        gender = list(df_sex[df_sex['ID']==subject]['Sex'])
        sex = int(gender[0]=='FEMALE')
             
        X1.append(list(features[:182]))
        X2.append(list(features[182:]))
        y_human.append(sex)

X1_human = np.array(X1)
X2_human = np.array(X2)

100%|██████████| 14465/14465 [00:12<00:00, 1144.69it/s]


In [7]:
print(X1_human.shape, X2_human.shape)

(10648, 182) (10648, 200)


In [8]:
df = pd.read_excel('/Users/jayantadey/Macaque.parcellated_thickness.xlsx')
df.head()

,Unnamed: 0,participant_id,age,sex,Markov.1,Markov.2,Markov.3,Markov.4,Markov.5,Markov.6,...,Schaefer217.191,Schaefer217.192,Schaefer217.193,Schaefer217.194,Schaefer217.195,Schaefer217.196,Schaefer217.197,Schaefer217.198,Schaefer217.199,Schaefer217.200
0,0,sub-1001,1.756164,M,3.048436,3.908286,3.221595,3.615675,4.662432,3.707754,...,4.231826,4.908868,4.522730,2.294943,2.853976,3.406234,4.261370,4.131977,3.387978,3.451267
1,1,sub-1002,1.783562,F,3.053520,3.748308,3.043567,3.764927,4.708283,4.060617,...,4.384853,4.849508,4.589500,2.443734,2.855187,3.344378,3.926697,3.477919,2.962553,3.474969
2,2,sub-1003,1.756164,M,3.211265,4.122524,3.374628,4.022762,4.759439,4.182558,...,4.570739,4.921833,4.770724,3.106145,3.094785,3.350355,4.562199,4.212585,3.582792,3.827813
3,3,sub-1004,1.756164,M,3.004275,3.681716,3.227427,3.762712,4.555942,3.984013,...,4.264869,4.935628,4.505048,3.337418,2.892611,3.690076,4.095378,4.328465,3.763171,3.758017
4,4,sub-1005,1.742466,M,2.868796,3.837011,2.997172,3.724171,4.537298,3.816082,...,4.154663,4.817727,4.695378,3.965287,3.219764,3.268439,4.115168,3.889531,3.271547,4.040183


In [9]:
df_sex = pd.read_csv('/Users/jayantadey/spmmouse_segment/uwmadison.csv')
df_sex.head()

,participant_id,age,sex
0,sub-1001,1.756164,M
1,sub-1002,1.783562,F
2,sub-1003,1.756164,M
3,sub-1004,1.756164,M
4,sub-1005,1.742466,M


In [10]:
X1 = []
X2 = []
y_monkey = []
IDs = set(df['participant_id'])
ref_IDs = set(df_sex['participant_id'])

for subject in tqdm(IDs):
    if subject in ref_IDs:
        features = np.array(df[df['participant_id']==subject]).reshape(-1)[4:]
        gender = list(df_sex[df_sex['participant_id']==subject]['sex'])
        sex = int(gender[0]=='F')
             
        X1.append(list(features[:182]))
        X2.append(list(features[182:]))
        y_monkey.append(sex)

X1_monkey = np.array(X1)
X2_monkey = np.array(X2)

100%|██████████| 592/592 [00:00<00:00, 3647.03it/s]


In [11]:
print(X1_monkey.shape, X2_monkey.shape)

(592, 182) (592, 200)


### Try random forest (trained on Humans, tested on monkeys)


# Markov

In [14]:
X1_human = np.nan_to_num(X1_human)
X2_human = np.nan_to_num(X2_human)
X1_monkey = np.nan_to_num(X1_monkey)
X2_monkey = np.nan_to_num(X2_monkey)

reps = 5
accuracy = 0.0

for ii in tqdm(range(reps)):
    x_train, x_test, y_train, y_test = train_test_split(
                    X1_human, y_human, train_size=0.8, random_state=ii, stratify=y_human)
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(x_train,y_train)
    accuracy += np.mean(clf.predict(X1_monkey)==y_monkey)

print('Accuracy is ',accuracy/reps)

100%|██████████| 5/5 [00:37<00:00,  7.52s/it]

Accuracy is  0.5162162162162162


# Schaefer

In [16]:
reps = 5
accuracy = 0.0

for ii in tqdm(range(reps)):
    x_train, x_test, y_train, y_test = train_test_split(
                    X2_human, y_human, train_size=0.8, random_state=ii, stratify=y_human)
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(x_train,y_train)
    accuracy += np.mean(clf.predict(X2_monkey)==y_monkey)

print('Accuracy is ',accuracy/reps)

100%|██████████| 5/5 [00:37<00:00,  7.60s/it]

Accuracy is  0.48040540540540544


In [17]:
np.mean(y_monkey)

0.44763513513513514

In [24]:
from proglearn.deciders import SimpleArgmaxAverage
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.transformers import (
    NeuralClassificationTransformer,
    TreeClassificationTransformer,
)
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter

In [26]:
default_transformer_class = TreeClassificationTransformer
default_transformer_kwargs = {
    "kwargs": {"max_depth": 30, "max_features": "sqrt"}
}

default_voter_class = TreeClassificationVoter
default_voter_kwargs = {}

default_decider_class = SimpleArgmaxAverage

progressive_learner = ProgressiveLearner(
    default_transformer_class=default_transformer_class,
    default_transformer_kwargs=default_transformer_kwargs,
    default_voter_class=default_voter_class,
    default_voter_kwargs=default_voter_kwargs,
    default_decider_class=default_decider_class,
)


In [27]:
y_human = np.array(y_human)
y_monkey = np.array(y_monkey)

x_train2, x_test2, y_train2, y_test2 = train_test_split(
                    X1_monkey, y_monkey, train_size=0.8, random_state=0, stratify=y_monkey)


progressive_learner.add_task(
            X=X1_human,
            y=y_human,
            task_id=0,
            num_transformers=1000,
            transformer_voter_decider_split=[0.67, 0.33, 0],
            decider_kwargs={
                "classes": np.unique(
                    y_human
                )
            },
        )

progressive_learner.add_task(
            X=x_train2,
            y=y_train2,
            task_id=1,
            num_transformers=1000,
            transformer_voter_decider_split=[0.67, 0.33, 0],
            decider_kwargs={
                "classes": np.unique(
                    y_train2
                )
            },
        )

In [28]:
multitask_label = progressive_learner.predict(x_test2, task_id=1)
singletask_label = progressive_learner.predict(x_test2, task_id=1, transformer_ids=[1])

print('Multi-task accuracy ', np.mean(multitask_label==y_test2))
print('Single-task accuracy ', np.mean(singletask_label==y_test2))

Multi-task accuracy  0.6050420168067226
Single-task accuracy  0.6470588235294118


In [29]:
progressive_learner = ProgressiveLearner(
    default_transformer_class=default_transformer_class,
    default_transformer_kwargs=default_transformer_kwargs,
    default_voter_class=default_voter_class,
    default_voter_kwargs=default_voter_kwargs,
    default_decider_class=default_decider_class,
)

x_train2, x_test2, y_train2, y_test2 = train_test_split(
                    X2_monkey, y_monkey, train_size=0.8, random_state=0, stratify=y_monkey)


progressive_learner.add_task(
            X=X2_human,
            y=y_human,
            task_id=0,
            num_transformers=1000,
            transformer_voter_decider_split=[0.67, 0.33, 0],
            decider_kwargs={
                "classes": np.unique(
                    y_human
                )
            },
        )

progressive_learner.add_task(
            X=x_train2,
            y=y_train2,
            task_id=1,
            num_transformers=1000,
            transformer_voter_decider_split=[0.67, 0.33, 0],
            decider_kwargs={
                "classes": np.unique(
                    y_train2
                )
            },
        )

In [30]:
multitask_label = progressive_learner.predict(x_test2, task_id=1)
singletask_label = progressive_learner.predict(x_test2, task_id=1, transformer_ids=[1])

print('Multi-task accuracy ', np.mean(multitask_label==y_test2))
print('Single-task accuracy ', np.mean(singletask_label==y_test2))

Multi-task accuracy  0.6638655462184874
Single-task accuracy  0.6554621848739496
